## Reloading

Mirar este issue: https://github.com/huggingface/transformers/issues/10204

In [1]:
import datasets
from datasets import load_dataset, Features, Value, load_from_disk

train_dataset = load_from_disk("../data/arrow/tokenized_train")
test_dataset = load_from_disk("../data/arrow/tokenized_test")

train_dataset, test_dataset

(Dataset({
     features: ['attention_mask', 'input_ids', 'special_tokens_mask', 'text', 'token_type_ids'],
     num_rows: 25600000
 }),
 Dataset({
     features: ['attention_mask', 'input_ids', 'special_tokens_mask', 'text', 'token_type_ids'],
     num_rows: 5215789
 }))

In [1]:
import torch
from transformers import AutoTokenizer
from finetune_vs_scratch.preprocessing import special_tokens
from transformers import BertForMaskedLM, BertTokenizerFast
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

tokenizer = BertTokenizerFast.from_pretrained(model_name)
tokenizer.model_max_length = 128
tokenizer.add_tokens(special_tokens)


4

In [2]:
tokenizer.save_pretrained("../data/beto-tokenizer")

('../data/beto-tokenizer/tokenizer_config.json',
 '../data/beto-tokenizer/special_tokens_map.json',
 '../data/beto-tokenizer/vocab.txt',
 '../data/beto-tokenizer/added_tokens.json',
 '../data/beto-tokenizer/tokenizer.json')

In [8]:
tweets = train_dataset[:2048*10]["text"]

In [11]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": tweets})

In [13]:

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_special_tokens_mask=True)
batch_size = 2048

dataset = dataset.map(tokenize, batch_size=batch_size, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [14]:

dataset.save_to_disk("../data/arrow/small_dataset")

In [15]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jmperez/projects/finetune_vs_scratch/notebooks


In [27]:

batch_size = 2048
steps = 12_500

needed_tweets = batch_size * steps

needed_tweets

25600000

In [22]:
import random

random.seed(2021)

idx = random.sample(list(range(len(train_dataset))), 10*batch_size)

train_dataset = train_dataset.select(idx)

In [23]:

len(train_dataset)

20480

In [24]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_special_tokens_mask=True)



train_dataset = train_dataset.map(tokenize, batch_size=batch_size, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [25]:
train_dataset.save_to_disk("prueba")